# 🎯 GÉNÉRATION DES PRÉDICTIONS KAGGLE

**Ce notebook**:
1. Charge les modèles entraînés
2. Prépare le test set (2012-2013) avec les mêmes features
3. Génère les prédictions
4. Crée le fichier submission.csv

**⚠️ Prérequis**: Avoir exécuté KAGGLE_TRAIN_MODEL.ipynb

In [1]:
# IMPORTS
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

print("✓ Imports réussis!")

✓ Imports réussis!


## 1. CHARGEMENT DES MODÈLES ENTRAÎNÉS

In [2]:
# Charger les modèles
print("📦 Chargement des modèles...")
with open('xgb_final.pkl', 'rb') as f:
    xgb_model = pickle.load(f)
with open('lgb_final.pkl', 'rb') as f:
    lgb_model = pickle.load(f)
with open('cat_final.pkl', 'rb') as f:
    cat_model = pickle.load(f)
with open('imputer.pkl', 'rb') as f:
    imputer = pickle.load(f)
with open('weights.pkl', 'rb') as f:
    weights = pickle.load(f)
with open('features.pkl', 'rb') as f:
    features = pickle.load(f)

print(f"✓ Modèles chargés!")
print(f"✓ Features: {len(features)}")
print(f"✓ Poids ensemble: {weights}")

📦 Chargement des modèles...
✓ Modèles chargés!
✓ Features: 26
✓ Poids ensemble: {'XGBoost': np.float64(0.32493732368191636), 'LightGBM': np.float64(0.3272881517152026), 'CatBoost': np.float64(0.3477745246028811)}


## 2. CHARGEMENT DU TEST SET

**Important**: On doit d'abord merger le test avec les données météo 2012-2013

In [17]:
# OPTION 1: Si vous avez déjà un fichier test_synop_merged.csv
df_test = pd.read_csv('data_plus/test_synop_merged.csv')

print (df_test.head())
print (df_test.columns)
print (df_test.info())

# OPTION 2: Charger le test brut et le merger avec les données météo
# (Code de merge à ajouter - similaire au train)

# Pour l'instant, on suppose que vous devez créer ce fichier
# print("⚠️ IMPORTANT:")
# print("Vous devez d'abord créer le test set avec les données météo 2012-2013")
# print("Voir le script de préparation du test ci-dessous...")

# PLACEHOLDER - À adapter selon votre structure
# df_test = pd.read_csv('data_origin/test.csv')
# Puis merger avec données météo 2012-2013 comme fait pour le train

     Id      week  region_code      region_name  week_year      tend  \
0  3235  201352.0           42           ALSACE     201352  3.571429   
1  3236  201352.0           72        AQUITAINE     201352 -1.694139   
2  3237  201352.0           83         AUVERGNE     201352  3.035714   
3  3238  201352.0           25  BASSE-NORMANDIE     201352  8.035714   
4  3239  201352.0           26        BOURGOGNE     201352  1.261905   

           dd        ff           t          td  ...     rafper   per  \
0  212.321429  4.460714  282.805357  279.551786  ...   8.400000 -10.0   
1  172.884615  4.378022  280.476282  277.717949  ...   8.811401 -10.0   
2  211.250000  4.996429  279.396429  277.405357  ...   8.851786 -10.0   
3  216.005411  7.758885  281.120833  278.380952  ...  13.909942 -10.0   
4  192.097403  5.424859  280.119762  275.674167  ...  10.343972 -10.0   

   ht_neige  ssfrai  perssfrai       rr1       rr3       rr6      rr12  \
0       0.0     0.0 -46.595745  0.346429  0.942857  1.

## 3. PRÉPARATION DU TEST SET

In [10]:
# Cette cellule suppose que df_test est chargé et a les colonnes météo
# Similaires à df_train_full

def prepare_test_set(df_test, df_train_full):
    """
    Préparer le test set avec les MÊMES features que le train
    """
    df_test = df_test.copy()
    
    # Convertir date
    # Créer date à partir de week_year si nécessaire
    def week_to_datetime(week_str):
        year = int(str(week_str)[:4])
        week = int(str(week_str)[4:6])
        from datetime import datetime
        jan4 = datetime(year, 1, 4)
        week_one_monday = jan4 - pd.Timedelta(days=jan4.weekday())
        return week_one_monday + pd.Timedelta(weeks=week-1)
    
    if 'date' not in df_test.columns:
        df_test['date'] = df_test['week'].apply(week_to_datetime)
    df_test['date'] = pd.to_datetime(df_test['date'])
    
    # Ajouter saison
    def get_season(date):
        month = date.month
        if month in [12, 1, 2]:
            return 'Hiver'
        elif month in [3, 4, 5]:
            return 'Printemps'
        elif month in [6, 7, 8]:
            return 'Ete'
        else:
            return 'Automne'
    
    df_test['saison'] = df_test['date'].apply(get_season)
    
    # Features temporelles
    df_test['year'] = df_test['date'].dt.year
    df_test['month'] = df_test['date'].dt.month
    df_test['week_of_year'] = df_test['date'].dt.isocalendar().week
    df_test['day_of_year'] = df_test['date'].dt.dayofyear
    
    # Features cycliques
    df_test['week_sin'] = np.sin(2 * np.pi * df_test['week_of_year'] / 52)
    df_test['week_cos'] = np.cos(2 * np.pi * df_test['week_of_year'] / 52)
    df_test['month_sin'] = np.sin(2 * np.pi * df_test['month'] / 12)
    df_test['month_cos'] = np.cos(2 * np.pi * df_test['month'] / 12)
    
    # Encoder saison
    saison_map = {'Hiver': 1, 'Printemps': 2, 'Ete': 3, 'Automne': 4}
    df_test['saison_encoded'] = df_test['saison'].map(saison_map)
    
    # 🎯 FEATURES HISTORIQUES - Calculées depuis le TRAIN
    # Créer les mappings depuis le train
    hist_week_map = df_train_full.groupby(['region_code', 'week_of_year'])['TauxGrippe'].mean().to_dict()
    hist_month_map = df_train_full.groupby(['region_code', 'month'])['TauxGrippe'].mean().to_dict()
    hist_season_map = df_train_full.groupby(['region_code', 'saison'])['TauxGrippe'].mean().to_dict()
    region_mean_map = df_train_full.groupby('region_code')['TauxGrippe'].mean().to_dict()
    region_std_map = df_train_full.groupby('region_code')['TauxGrippe'].std().to_dict()
    week_global_map = df_train_full.groupby('week_of_year')['TauxGrippe'].mean().to_dict()
    
    # Appliquer au test
    df_test['TauxGrippe_hist_week_mean'] = df_test.apply(
        lambda x: hist_week_map.get((x['region_code'], x['week_of_year']), region_mean_map.get(x['region_code'], 0)),
        axis=1
    )
    df_test['TauxGrippe_hist_month_mean'] = df_test.apply(
        lambda x: hist_month_map.get((x['region_code'], x['month']), region_mean_map.get(x['region_code'], 0)),
        axis=1
    )
    df_test['TauxGrippe_hist_season_mean'] = df_test.apply(
        lambda x: hist_season_map.get((x['region_code'], x['saison']), region_mean_map.get(x['region_code'], 0)),
        axis=1
    )
    df_test['TauxGrippe_region_mean'] = df_test['region_code'].map(region_mean_map)
    df_test['TauxGrippe_region_std'] = df_test['region_code'].map(region_std_map)
    df_test['TauxGrippe_week_global_mean'] = df_test['week_of_year'].map(week_global_map)
    
    return df_test

print("✓ Fonction de préparation du test définie")

✓ Fonction de préparation du test définie


In [ ]:
# Charger le train pour les stats historiques
df_train_full = pd.read_csv('data_plus/train_synop_cleaned_complet.csv')
df_train_full['date'] = pd.to_datetime(df_train_full['date'])

# IMPORTANT: Créer les features temporelles sur le train AVANT de l'utiliser
print("📊 Création des features temporelles sur le train...")

# Features temporelles
df_train_full['year'] = df_train_full['date'].dt.year
df_train_full['month'] = df_train_full['date'].dt.month
df_train_full['week_of_year'] = df_train_full['date'].dt.isocalendar().week

print(f"✓ Train préparé: {df_train_full.shape}")

# Charger le test set
df_test = pd.read_csv('data_plus/test_synop_merged.csv')

# Préparer le test avec les mêmes transformations
df_test = prepare_test_set(df_test, df_train_full)

print(f"✓ Test set préparé: {df_test.shape}")
print(f"Colonnes test: {df_test.columns.tolist()}")

## 4. GÉNÉRATION DES PRÉDICTIONS

In [15]:
# Cette cellule suppose que df_test est préparé

# Préparer X_test
X_test = df_test[features]
test_ids = df_test['Id']

# Imputer
X_test = pd.DataFrame(
    imputer.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

print(f"✓ Test set préparé: {X_test.shape}")

# Prédictions avec les 3 modèles
print("\n🚀 Génération des prédictions...")
y_pred_xgb = xgb_model.predict(X_test)
print("✓ XGBoost")

y_pred_lgb = lgb_model.predict(X_test)
print("✓ LightGBM")

y_pred_cat = cat_model.predict(X_test)
print("✓ CatBoost")

# Ensemble (moyenne pondérée)
y_pred = (
    weights['XGBoost'] * y_pred_xgb +
    weights['LightGBM'] * y_pred_lgb +
    weights['CatBoost'] * y_pred_cat
)
print("✓ Ensemble")

# S'assurer que les prédictions sont positives
y_pred = np.maximum(y_pred, 0)

print(f"\n📊 Statistiques des prédictions:")
print(f"   Min: {y_pred.min():.2f}")
print(f"   Max: {y_pred.max():.2f}")
print(f"   Moyenne: {y_pred.mean():.2f}")
print(f"   Médiane: {np.median(y_pred):.2f}")

KeyError: "['week_of_year', 'month', 'week_sin', 'week_cos', 'month_sin', 'month_cos', 'saison_encoded', 'TauxGrippe_hist_week_mean', 'TauxGrippe_hist_month_mean', 'TauxGrippe_hist_season_mean', 'TauxGrippe_region_mean', 'TauxGrippe_region_std', 'TauxGrippe_week_global_mean'] not in index"

## 5. CRÉATION DU FICHIER SUBMISSION

In [ ]:
# Créer le fichier submission
submission = pd.DataFrame({
    'Id': test_ids,
    'TauxGrippe': y_pred
})

# Sauvegarder
submission.to_csv('submission_ensemble.csv', index=False)

print("\n✅ FICHIER SUBMISSION CRÉÉ!")
print("="*80)
print(f"Fichier: submission_ensemble.csv")
print(f"Lignes: {len(submission)}")
print(f"\nAperçu:")
print(submission.head(10))
print(f"\n🎯 Prêt à soumettre sur Kaggle!")
print("="*80)

## 6. (OPTIONNEL) CRÉER DES SUBMISSIONS INDIVIDUELLES

In [ ]:
# Si vous voulez tester les modèles individuellement

# XGBoost
submission_xgb = pd.DataFrame({'Id': test_ids, 'TauxGrippe': np.maximum(y_pred_xgb, 0)})
submission_xgb.to_csv('submission_xgb.csv', index=False)

# LightGBM
submission_lgb = pd.DataFrame({'Id': test_ids, 'TauxGrippe': np.maximum(y_pred_lgb, 0)})
submission_lgb.to_csv('submission_lgb.csv', index=False)

# CatBoost
submission_cat = pd.DataFrame({'Id': test_ids, 'TauxGrippe': np.maximum(y_pred_cat, 0)})
submission_cat.to_csv('submission_cat.csv', index=False)

print("✓ Submissions individuelles créées:")
print("  - submission_xgb.csv")
print("  - submission_lgb.csv")
print("  - submission_cat.csv")